In [1]:
import numpy as np
import xarray as xr
import sys
import awkward._v2 as ak
sys.path.insert(0, '../')
from clouddrift import dataformat, select

In [19]:
drifter_id = [1, 2, 3]
rowsize = [5, 4, 2]
longitude = [[-121, -111, 51, 61, 71], [12, 22, 32, 42], [103, 113]]
latitude = [[-90, -45, 45, 90, 0], [10, 20, 30, 40], [10, 20]]
t = [[1,2,3,4,5], [2,3,4,5], [4,5]]
ids = [[1,1,1,1,1], [2,2,2,2], [3,3]]
test = [[True, True, True, False, False], [True, False, False, False], [False, False]]
nb_obs = np.sum(rowsize)
nb_traj = len(drifter_id)
attrs_global = {
    "title": "test trajectories",
    "history": "version xyz",
}
variables_coords = {"ids": "ids", "time": "t", "lon": "longitude", "lat": "latitude"}
coords = {
    "longitude": longitude, 
    "latitude": latitude,
    "ids":  ids,
    "t": t
}
metadata = {"ID": drifter_id, "rowsize": rowsize}
data = {"test": test}

# append xr.Dataset to a list
list_ds = []
for i in range(0, len(rowsize)):
    xr_coords = {}
    for var in coords.keys():
        xr_coords[var] = (
            ["obs"],
            coords[var][i],
            {"long_name": f"variable {var}", "units": "-"},
        )

    xr_data = {}
    for var in metadata.keys():
        xr_data[var] = (
            ["traj"],
            [metadata[var][i]],
            {"long_name": f"variable {var}", "units": "-"},
        )

    for var in data.keys():
        xr_data[var] = (
            ["obs"],
            data[var][i],
            {"long_name": f"variable {var}", "units": "-"},
        )

    list_ds.append(
        xr.Dataset(coords=xr_coords, data_vars=xr_data, attrs=attrs_global)
    )

In [20]:
# create test ragged array
ra = dataformat.create_ragged_array(
    [0, 1, 2], lambda i: list_ds[i], variables_coords, ["ID", "rowsize"], ["test"]
)

ds = ra.to_awkward()

Filling the ragged array: 100%|████████████████████████████████████████████████| 3/3 [00:00<00:00, 2141.41it/s]


In [39]:
dd =select.subset(ds, {"ID": [1,2], "lat": (-90,20), "lon": (-180, 25), "test": True})

In [40]:
dd.ID

<Array [1, 2] type='2 * int64[parameters={"attrs": {"long_name": "variable ...'>

In [41]:
dd.obs.lon

<Array [[-121, -111], [12]] type='2 * var * int64'>

In [42]:
dd.obs.lat

<Array [[-90, -45], [10]] type='2 * var * int64'>

In [43]:
dd.obs.test

<Array [[True, True], [True]] type='2 * var * bool'>

# Global Drifter Program (GDP)

As part of this Notebook, we will use the *GDP historical dataset* to highlight the required steps to preprocess and dataset into a format that can be ingest by the *CloudDrift* library.

## Dataformat module

The `dataformat.py` module contains the class `create_ragged_array` to transform a series of archives into an *Awkward Array* where all variables is stored as a *ragged array*. The module also contains `read_from_netcdf` and `read_from_parquet` to initialize the *Awkward Array* directly from an previously preprocessed archive. Right now, it *only* supports local array but we will soon add the possibility of *lazy-loading* array stored in the Cloud.

In [1]:
import sys
sys.path.insert(0, '../')
from clouddrift import dataformat

The main class of this module is *create_ragged_array* and is used to create a single archive that can be saved to a netCDF or Parquet file. The signature of the class is:

In [2]:
dataformat.create_ragged_array?

Init signature:
dataformat.create_ragged_array(
    indices: list,
    preprocess_func: collections.abc.Callable[[int], xarray.core.dataset.Dataset],
    vars_coords: dict,
    vars_meta: list = [],
    vars_data: list = [],
    rowsize_func: collections.abc.Callable[[int], int] = None,
)
Docstring:      <no docstring>
File:           ~/Library/CloudStorage/OneDrive-FloridaStateUniversity/projects/clouddrift/clouddrift/dataformat.py
Type:           type
Subclasses:     


## Dataset-specific functions

Since each dataset is different, we have to create specific functions to preprocess the dataset (`preprocess_func`) and return the metadata and data of a single trajectory. This was inspired by the [Pangeo Forge](https://pangeo-forge.readthedocs.io/en/latest/) project. The class *create_ragged_array* will use those functions to create the single archive of ragged arrays. More precisely, it requires:
- a list of indices (or identification number) that will be concatenate into the ragged array format
- a preprecessing function with the following signature:
    - `Signature: preprocess_func(index: int) -> xarray.core.dataset.Dataset`, where the index parameter is an identifier of a trajectory, e.g. the identification number of an Argo float) and returns an *xarray Dataset*. 
- a dictionnary mapping the mandatory coordinates list to the name of those variables in the dataset, e.g.
    coords = {'ids': 'number', 'time': 't', 'longitude': 'lon', 'latitude': 'lat'}
- an optional list of variable names containing metadata information about the trajectory (size: 1 per trajectory)
- an optional list of variable names containing the data along the trajectory (size: number of observations per trajectory)
- an optional funcition that returns directly the number of observation of a trajectory (`Signature: rowsize_func(index: int) -> int`)
    
This function can performs all type of operations, such as formatting the date, changing the type of variables, modifying the metadata, etc. We provide preprocessing function for different datasets in the `data/recipes/` folder. The class also needs to *initially* calculate the sum of all observations. By default, this is performed using `lambda i: preprocess_func(i).dims['obs']`. To *speed up* this process, in the situation where a lot of preprocessing are performed, it is possible to provide a second function `rowsize_func`, that returns directly the number of observation of a trajectory (`Signature: rowsize_func(index: int) -> int`)

Finally, we included an function to download (or update) the GDP dataset.

In [3]:
from data import gdp6h

Listing AOML directory...
https://www.aoml.noaa.gov/ftp/pub/phod/lumpkin/netcdf/buoydata_1_5000
https://www.aoml.noaa.gov/ftp/pub/phod/lumpkin/netcdf/buoydata_5001_10000
https://www.aoml.noaa.gov/ftp/pub/phod/lumpkin/netcdf/buoydata_10001_15000
https://www.aoml.noaa.gov/ftp/pub/phod/lumpkin/netcdf/buoydata_15001_dec20


# Download

The download function will store the raw dataset into the `data/raw/` folder specified in the `gdp.py` module. By default `download_gdp_data()` will download the complete GPD dataset (containing 17,324 files as of May 2022) from the AOML `https` server.

**Note**: this Notebook is very similar to the `data-glad.ipynb` Notebook because very few functions have to be created to transform a new dataset. We hope that this will encourage people to use this dataformat and utilize the CloudDrift library.

In [4]:
gdp6h.download?

Signature: gdp6h.download(drifter_ids: list = None, n_random_id: int = None)
Docstring:
Download individual netCDF files from the AOML server

:param drifter_ids [list]: list of drifter to retrieve (Default: all)
:param n_random_id [int]: randomly select n drifter netCDF files
:return drifters_ids [list]: list of retrived drifter
File:      ~/Library/CloudStorage/OneDrive-FloridaStateUniversity/projects/clouddrift/data/gdp6h.py
Type:      function


It is possible to prodive a list of `drifter_ids` to retrieve a subset and/or specified a integer `n_random_id` to randomly retrieve `n` trajectory. The function returns the list of `drifters_ids` that was downloaded, and can be passed to create the ragged array.

In [5]:
drifter_ids = gdp6h.download(n_random_id=100)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 77172.11it/s]


Once the data downloaded, it is possible to create the ragged array and either save a netCDF, parquet file, or simply output an Awkward Array that can be used for analysis.

In [6]:
drifter_ids[:5]

[14832, 26654, 30347, 35262, 39135]

In [7]:
coords = {'ids': 'ids', 'time': 'time', 'lon': 'longitude', 'lat': 'latitude'}
metadata = ['ID', 'rowsize', 'WMO', 'expno', 'deploy_date', 'deploy_lat', 'deploy_lon', 'end_date', 'end_lat', 'end_lon', 'drogue_lost_date', 'typedeath', 'typebuoy', 'DeployingShip', 'DeploymentStatus', 'BuoyTypeManufacturer', 'BuoyTypeSensorArray', 'CurrentProgram', 'PurchaserFunding', 'SensorUpgrade', 'Transmissions', 'DeployingCountry', 'DeploymentComments', 'ManufactureYear', 'ManufactureMonth', 'ManufactureSensorType', 'ManufactureVoltage', 'FloatDiameter', 'SubsfcFloatPresence', 'DrogueType', 'DrogueLength', 'DrogueBallast', 'DragAreaAboveDrogue', 'DragAreaOfDrogue', 'DragAreaRatio', 'DrogueCenterDepth', 'DrogueDetectSensor']
data = ['ve', 'vn', 'temp', 'err_lat', 'err_lon', 'err_temp', 'drogue_status']

ra = dataformat.create_ragged_array(drifter_ids,
                         gdp6h.preprocess,
                         coords, 
                         metadata, 
                         data,
                         rowsize_func=gdp6h.rowsize
                        )

Calculating the number of observations: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 83.84it/s]
/Users/pmiron/miniforge3/envs/clouddrift/lib/python3.9/site-packages/xarray/coding/variables.py:140: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv
Filling the ragged array: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 36.16it/s]


## Export

In [8]:
ra.to_parquet('../data/process/gdp_6h.parquet')
ra.to_netcdf('../data/process/gdp_6h.nc')

## Import

In [9]:
ds2 = dataformat.read_from_parquet('../data/process/gdp_6h.parquet')

In [10]:
ds2.ID

<Array [14832, 26654, ..., 67242750, 68240990] type='100 * int64[parameters...'>

## Awkward Array

In [11]:
ds = ra.to_awkward()

In [12]:
ds.ID

<Array [14832, 26654, ..., 67242750, 68240990] type='100 * int64[parameters...'>

In [13]:
ds.fields

['ID',
 'rowsize',
 'WMO',
 'expno',
 'deploy_date',
 'deploy_lat',
 'deploy_lon',
 'end_date',
 'end_lat',
 'end_lon',
 'drogue_lost_date',
 'typedeath',
 'typebuoy',
 'DeployingShip',
 'DeploymentStatus',
 'BuoyTypeManufacturer',
 'BuoyTypeSensorArray',
 'CurrentProgram',
 'PurchaserFunding',
 'SensorUpgrade',
 'Transmissions',
 'DeployingCountry',
 'DeploymentComments',
 'ManufactureYear',
 'ManufactureMonth',
 'ManufactureSensorType',
 'ManufactureVoltage',
 'FloatDiameter',
 'SubsfcFloatPresence',
 'DrogueType',
 'DrogueLength',
 'DrogueBallast',
 'DragAreaAboveDrogue',
 'DragAreaOfDrogue',
 'DragAreaRatio',
 'DrogueCenterDepth',
 'DrogueDetectSensor',
 'obs']

In [14]:
ds.obs.fields

['ids',
 'time',
 'lon',
 'lat',
 've',
 'vn',
 'temp',
 'err_lat',
 'err_lon',
 'err_temp',
 'drogue_status']